In [ ]:
!pip install selenium beautifulsoup4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.14.0
    Uninstalling typing_extensions-4.14.0:
      Successfully uninstalled typing_extensions-4.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 r

In [ ]:
#bucar propostas
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

# Configurações do Selenium
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--window-size=1920,1080')
options.add_argument('--lang=pt-BR')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36')

driver = webdriver.Chrome(options=options)
driver.get('https://educacao.uol.com.br/bancoderedacoes/')
time.sleep(3)

# Clica no botão "Ver mais" até que ele desapareça (carregou tudo)
while True:
    try:
        ver_mais = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.btn-more'))
        )
        driver.execute_script("arguments[0].click();", ver_mais)
        time.sleep(2)  # esperar o carregamento do novo conteúdo
    except:
        break  # Não há mais botão ou não está visível

# Pega o conteúdo final
soup = BeautifulSoup(driver.page_source, 'html.parser')

propostas = []
for a in soup.select('a[href*="/bancoderedacoes/propostas/"]'):
    titulo_element = a.select_one('h3.thumb-title')
    if titulo_element:
        titulo = titulo_element.get_text(strip=True)
        url = a['href']
        if not url.startswith('http'):
            url = 'https://educacao.uol.com.br' + url
        propostas.append({'titulo': titulo, 'url': url})

driver.quit()

# Exibir resultados
print(f"📌 {len(propostas)} propostas encontradas:")
for proposta in propostas:
    print(f"- {proposta['titulo']} -> {proposta['url']}")


📌 55 propostas encontradas:
- Qualificação e o futuro do emprego -> https://educacao.uol.com.br/bancoderedacoes/propostas/qualificacao-e-o-futuro-do-emprego.htm
- Supremo Tribunal Federal e opinião pública -> https://educacao.uol.com.br/bancoderedacoes/propostas/supremo-tribunal-federal-e-opiniao-publica.htm
- Ciência, tecnologia e superação dos limites humanos -> https://educacao.uol.com.br/bancoderedacoes/propostas/ciencia-tecnologia-e-superacao-dos-limites-humanos.htm
- Um réu deve ou não ser preso após a condenação em 2ª. instância? -> https://educacao.uol.com.br/bancoderedacoes/propostas/um-reu-deve-ou-nao-ser-preso-apos-a-condenacao-em-2-instancia.htm
- Agrotóxicos ou defensivos agrícolas: dois nomes e uma polêmica -> https://educacao.uol.com.br/bancoderedacoes/propostas/agrotoxicos-ou-defensivos-agricolas-dois-nomes-e-uma-polemica.htm
- Preservação da Amazônia: Desafio brasileiro ou internacional? -> https://educacao.uol.com.br/bancoderedacoes/propostas/preservacao-da-amazonia-d

In [ ]:
#indo além: propostas
"""
Scraper de Redações do Banco de Redações – múltiplas propostas
--------------------------------------------------------------
• Para cada URL de proposta (lista PROPOSTA_URLS) o script coleta todas as redações,
  extrai corpo, comentário geral, competências e notas, e grava em um HTML único.
• Requisitos: selenium, beautifulsoup4, chromedriver compatível no PATH.
"""

import re
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

PROPOSTA_URLS = [
    'https://educacao.uol.com.br/bancoderedacoes/propostas/qualificacao-e-o-futuro-do-emprego.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/supremo-tribunal-federal-e-opiniao-publica.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/ciencia-tecnologia-e-superacao-dos-limites-humanos.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/um-reu-deve-ou-nao-ser-preso-apos-a-condenacao-em-2-instancia.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/agrotoxicos-ou-defensivos-agricolas-dois-nomes-e-uma-polemica.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/preservacao-da-amazonia-desafio-brasileiro-ou-internacional.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/criptomoeda-tecnologia-e-revolucao-economica.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/a-ciencia-na-era-da-pos-verdade.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/universidade-em-crise-quem-paga-a-conta.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/a-fe-e-decisiva-para-uma-vida-melhor.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/os-ursos-polares-da-russia-e-um-dilema-ecologico.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/cantar-ou-nao-cantar-o-hino-nacional-eis-a-questao.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/posse-de-armas-mais-seguranca-ou-mais-perigo.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/o-brasil-e-os-imigrantes-no-mundo-contemporaneo.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/epidemia-alimentar-sobrepeso-e-obesidade.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/a-onda-conservadora-e-o-brasil-nos-proximos-anos.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/patrimonio-historico-e-cultural-brasileiro-um-caso-de-descaso.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/direitos-humanos-em-defesa-de-quem.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/redes-sociais-e-manipulacao-politica.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/por-que-os-jovens-querem-deixar-o-brasil.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/o-brasil-paralisado-o-que-voce-pensa-sobre-a-greve-dos-caminhoneiros.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/o-direito-ao-foro-privilegiado.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/informacao-no-rotulo-de-produtos-transgenicos.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/violencia-e-drogas-o-papel-do-usuario.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/pena-de-morte-o-brasil-deve-adota-la-por-que.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/a-arte-tem-o-direito-de-questionar-tudo.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/as-redes-sociais-ajudam-ou-dificultam-as-relacoes-entre-as-pessoas.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/delacao-premiada-justica-injustica-ou-traicao.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/a-terapia-de-reversao-da-orientacao-sexual.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/como-melhorar-a-educacao-sem-valorizar-o-professor.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/brasileiros-tem-pessima-educacao-argumentativa-segundo-cientista.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/por-que-nao-ha-novas-manifestacoes-nas-ruas.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/internacao-compulsoria-de-dependentes-de-crack.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/perigos-do-universo-digital.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/terceirizacao-avanco-ou-retrocesso.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/direitos-em-conflito-liberdade-de-expressao-e-intimidade.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/e-preferivel-praticar-ou-sofrer-uma-injustica.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/somos-todos-corruptos.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/pos-verdade-opiniao-publica-e-democracia.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/filantropia-um-exemplo-a-ser-seguido.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/artes-e-educacao-fisica-opcionais-ou-obrigatorias.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/voce-faz-parte-da-turma-do-eu-me-acho.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/escola-no-brasil-com-partido-ou-sem-partido.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/escravizar-e-humano.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/estupro-como-prevenir-esse-crime.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/politica-x-ciencia-a-pilula-do-cancer.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/impeachment-a-presidente-deve-perder-o-mandato.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/carta-convite-discutir-discriminacao-na-escola.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/a-tecnologia-e-a-eliminacao-de-empregos.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/por-que-o-brasil-nao-consegue-vencer-o-aedes-aegypti.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/forma-fisica-corpo-perfeito-e-consumismo.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/disciplina-ordem-e-autoridade-favorecem-a-educacao.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/o-sucesso-vem-da-escola-ou-do-esforco-individual.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/bandido-bom-e-bandido-morto.htm',
    'https://educacao.uol.com.br/bancoderedacoes/propostas/mariana-fatalidade-ou-negligencia.htm'
]

# ----------------------------------------------------------------------
# Funções auxiliares
# ----------------------------------------------------------------------
def clean_filename(s: str) -> str:
    """Remove caracteres inválidos para nomes de arquivo (Windows/Unix)."""
    return re.sub(r'[\\/:"*?<>|]+', '', s)


def setup_driver() -> webdriver.Chrome:
    """Configura o Chrome headless."""
    opts = Options()
    opts.add_argument('--headless')
    opts.add_argument('--disable-gpu')
    opts.add_argument('--window-size=1920,1080')
    opts.add_argument('--lang=pt-BR')
    opts.add_argument('--no-sandbox')
    opts.add_argument('--disable-dev-shm-usage')
    opts.add_argument(
        'user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
        'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
    )
    return webdriver.Chrome(options=opts)


def extrair_competencias_e_notas(page_soup):
    """Retorna (html_lista_competencias, html_tabela_notas)."""
    artigo = page_soup.find('article', class_='rt-body')
    if not artigo:
        return '', ''

    competencias, notas = [], []
    for linha in artigo.find_all('div', class_='rt-line-option'):
        tema, pontos = linha.find('span', class_='topic'), linha.find('span', class_='points')
        if tema and pontos:
            competencias.append(tema.get_text(strip=True))
            notas.append(pontos.get_text(strip=True))

    html_comp = '<h3>Competências Avaliadas:</h3><ul>'
    html_comp += ''.join(f'<li>{c}</li>' for c in competencias[:-1])  # ignora "Nota final"
    html_comp += '</ul>'

    html_tab = (
        '<h3>Notas por Competência:</h3>'
        '<table border="1" cellpadding="5" cellspacing="0">'
        '<tr><th>Competência</th><th>Nota</th></tr>'
        + ''.join(f'<tr><td>{c}</td><td>{n}</td></tr>' for c, n in zip(competencias, notas))
        + '</table>'
    )
    return html_comp, html_tab


def extrair_comentarios_gerais(page_soup):
    """Devolve bloco HTML do comentário geral + lista de observações."""
    bloco = page_soup.find('div', class_='text')
    return str(bloco) if bloco else ''


# ----------------------------------------------------------------------
# Lógica principal
# ----------------------------------------------------------------------
def main():
    driver = setup_driver()
    saida_html = 'redacoes_completas.html'

    with open(saida_html, 'w', encoding='utf-8') as f:
        # Cabeçalho do documento final
        f.write(
            """<!DOCTYPE html>
<html lang="pt-BR">
<head>
<meta charset="UTF-8">
<title>Redações completas</title>
<style>
 body{font-family:Arial,Helvetica,sans-serif;margin:20px;}
 h1{border-bottom:2px solid #444;padding-bottom:5px;}
 h2{color:#003366;margin-top:40px;}
 h3{color:#005580;}
 table{border-collapse:collapse;margin-bottom:40px;}
 th,td{border:1px solid #999;padding:8px;}
 section{margin-bottom:60px;border-bottom:1px dashed #ccc;padding-bottom:40px;}
 .comentarios{background:#f9f9f9;padding:10px 20px;border-left:4px solid #0077cc;margin-top:20px;}
</style>
</head><body>
<h1>Redações e Avaliações – Banco de Redações (UOL)</h1>
"""
        )

        # ------------------------------------------------------------------
        # Para evitar duplicação quando a mesma redação aparece em mais de
        # uma proposta, acumulamos URLs já processadas.
        # ------------------------------------------------------------------
        urls_processadas = set()

        for prop_idx, proposta_url in enumerate(PROPOSTA_URLS, 1):
            print(f'\n=== Proposta {prop_idx}/{len(PROPOSTA_URLS)} === {proposta_url}')
            try:
                driver.get(proposta_url)
                time.sleep(3)
            except Exception as e:
                print(f'⚠️ Não consegui abrir a proposta: {e}')
                continue

            proposta_soup = BeautifulSoup(driver.page_source, 'html.parser')

            # Título da proposta (H1 da página)
            titulo_proposta = proposta_soup.find('h1')
            titulo_proposta = titulo_proposta.get_text(strip=True) if titulo_proposta else proposta_url

            # Escreve cabeçalho da proposta no HTML de saída
            f.write(f'<h2>{titulo_proposta}</h2>\n')

            # ---- coleta links das redações dessa proposta -----------------
            redacoes = []
            for link in proposta_soup.find_all('a', href=True):
                href = link['href']
                if '/redacoes/' in href:
                    topic_span = link.find('span', class_='topic')
                    titulo_red = topic_span.get_text(strip=True) if topic_span else ''
                    full_url = href if href.startswith('http') else 'https://educacao.uol.com.br' + href
                    redacoes.append({'titulo': titulo_red, 'url': full_url})

            print(f'→ {len(redacoes)} redações encontradas nesta proposta.')

            # -------- percorre cada redação --------------------------------
            for idx, red in enumerate(redacoes, 1):
                if red['url'] in urls_processadas:
                    print(f'   • (pulada – já capturada) {red["url"]}')
                    continue
                print(f'   • [{idx}/{len(redacoes)}] Capturando redação: {red["url"]}')
                try:
                    driver.get(red['url'])
                    time.sleep(3)
                    page_soup = BeautifulSoup(driver.page_source, 'html.parser')

                    # Título real da redação
                    titulo_redacao = red['titulo'] or page_soup.find('h1').get_text(strip=True)

                    corpo_div = page_soup.find('div', class_='text-composition')
                    if not corpo_div:
                        print('     ⚠️ Corpo da redação não encontrado.')
                        continue
                    corpo_html = str(corpo_div)

                    comentarios_html = extrair_comentarios_gerais(page_soup)
                    if comentarios_html:
                        comentarios_html = f'<div class="comentarios">{comentarios_html}</div>'

                    comp_html, tabela_html = extrair_competencias_e_notas(page_soup)

                    # ------------ grava seção da redação --------------------
                    f.write('<section>\n')
                    f.write(f'<h3>{titulo_redacao}</h3>\n')
                    f.write(corpo_html + '\n')
                    f.write(comentarios_html + '\n')
                    f.write(comp_html + '\n')
                    f.write(tabela_html + '\n')
                    f.write('</section>\n')

                    urls_processadas.add(red['url'])
                    print('     ✅ adicionada')

                except Exception as e:
                    print(f'     ❌ Erro: {e}')

        # fecha documento HTML
        f.write('</body></html>')

    driver.quit()
    print(f'\n✅ Concluído! Resultado salvo em: {saida_html}')


if __name__ == '__main__':
    main()



=== Proposta 1/55 === https://educacao.uol.com.br/bancoderedacoes/propostas/qualificacao-e-o-futuro-do-emprego.htm
→ 20 redações encontradas nesta proposta.
   • [1/20] Capturando redação: https://educacao.uol.com.br/bancoderedacoes/redacoes/qualificacoes-para-o-mercado-de-trabalho.htm
     ✅ adicionada
   • [2/20] Capturando redação: https://educacao.uol.com.br/bancoderedacoes/redacoes/qualificacao-sera-o-futuro-do-emprego.htm
     ✅ adicionada
   • [3/20] Capturando redação: https://educacao.uol.com.br/bancoderedacoes/redacoes/o-futuro-dos-desempregados-no-brasil.htm
     ✅ adicionada
   • [4/20] Capturando redação: https://educacao.uol.com.br/bancoderedacoes/redacoes/nacoes-unidas-contra-o-desemprego.htm
     ✅ adicionada
   • [5/20] Capturando redação: https://educacao.uol.com.br/bancoderedacoes/redacoes/a-era-da-reinvencao-profissional.htm
     ✅ adicionada
   • [6/20] Capturando redação: https://educacao.uol.com.br/bancoderedacoes/redacoes/peao-tambem-e-gente.htm
     ✅ adiciona